# API и XML

Разные сайты предлагают информацию, которую может легко обрабатывать не только человек, но и другая программа. Это называется API — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером.

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). 

Для обработки XML-файлов можно использовать пакет Beautiful Soup. Единственное различие — нужно указать дополнительный параметр feautres="xml" при вызове функции BeautifulSoup — чтобы он не искал в документе HTML-теги.

In [1]:
from bs4 import BeautifulSoup

In [2]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [3]:
obj = BeautifulSoup(group, features="xml")
print(obj.prettify())

<?xml version="1.0" encoding="utf-8"?>
<group>
 <number>
  134
 </number>
 <student>
  <firstname>
   Виталий
  </firstname>
  <lastname>
   Иванов
  </lastname>
 </student>
 <student>
  <firstname>
   Мария
  </firstname>
  <lastname>
   Петрова
  </lastname>
 </student>
</group>



In [4]:
obj.group.number.string #найти номер группы

'134'

In [5]:
#перечислить всех студентов
for student in obj.group.findAll('student'):
    print(student.lastname.string, student.firstname.string)

Иванов Виталий
Петрова Мария


При работе с API мы обращаемся к специальной веб-странице по специальному адресу и передаём запрос.

## Пример: список статей из категории в Википедии

Допустим, нам потребовалось получить список всех статей из некоторой категории в Википедии. Мы могли бы открыть эту категорию в браузере и дальше использовать обычный веб-скреппинг. Однако разработчики Википедии сделали удобное API https://www.mediawiki.org/wiki/API:Tutorial. Чтобы научиться с ним работать, придётся познакомиться с документацией (так будет с любым API). 
Пример: https://www.mediawiki.org/wiki/API:Categorymembers


Взаимодействие с сервером при помощи API происходит с помощью отправки специальным образом сформированных запросов и получения ответа в одном из машинночитаемых форматов. Сейчас нас будет интересовать формат XML. 

Например, можно отправить такой запрос:

https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:Physics&cmsort=timestamp&cmdir=desc&format=xmlfm

Строка https://en.wikipedia.org/w/api.php (до знака вопроса) — это точка входа в API. Всё, что идёт после знака вопроса — это, собственно, запрос. Он представляет собой что-то вроде словаря и состоит из пар «ключ=значение», разделяемых амперсандом &. Некоторые символы приходится кодировать специальным образом.

Например, в адресе выше сказано, что мы хотим сделать запрос (action=query), перечислить элементы категории list=categorymembers, в качестве категории, которая нас интересует, указана Category:Physics (cmtitle=Category:Physics) и указаны некоторые другие параметры. 


In [6]:
import requests 

In [7]:
url = "https://en.wikipedia.org/w/api.php"
myparams = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'xml'
}
# список параметров передаем в виде обычного словаря

g = requests.get(url, params=myparams)

In [8]:
g.ok

True

In [9]:
mydata = BeautifulSoup(g.text, features='xml')

In [10]:
print(mydata.prettify())

<?xml version="1.0" encoding="utf-8"?>
<api batchcomplete="">
 <continue cmcontinue="page|44522e40322a503a4644010e01dc0d|1996857" continue="-||"/>
 <query>
  <categorymembers>
   <cm ns="0" pageid="22939" title="Physics"/>
   <cm ns="0" pageid="844186" title="Modern physics"/>
   <cm ns="100" pageid="1653925" title="Portal:Physics"/>
   <cm ns="0" pageid="78053369" title="Bijel"/>
   <cm ns="0" pageid="74985603" title="Edge states"/>
   <cm ns="0" pageid="78147827" title="Electrostatic solitary wave"/>
   <cm ns="0" pageid="78554064" title="History of the LED"/>
   <cm ns="0" pageid="78751748" title="Missile lofting"/>
   <cm ns="0" pageid="168907" title="Naïve physics"/>
   <cm ns="0" pageid="78245824" title="Nottingham effect"/>
  </categorymembers>
 </query>
</api>



In [11]:
#находим все вхождения тега <cm> и выведем их атрибут title:
for cm in mydata("cm"):
    print(cm['title'])

Physics
Modern physics
Portal:Physics
Bijel
Edge states
Electrostatic solitary wave
History of the LED
Missile lofting
Naïve physics
Nottingham effect


По умолчанию сервер вернул нам список из 10 элементов. Если мы хотим больше, нужно воспользоваться элементом continue — это своего рода гиперссылка на следующие 10 элементов. Или можно задать значение параметра cmlimit (до 500 записей).

In [12]:
mydata.find("continue")['cmcontinue']

'page|44522e40322a503a4644010e01dc0d|1996857'

In [13]:
myparams['cmcontinue'] = mydata.api("continue")[0]['cmcontinue']

In [14]:
g = requests.get(url, params=myparams)
mydata = BeautifulSoup(g.text, features='xml')
for cm in mydata("cm"):
    print(cm['title'])



Nucleation
Perfect fluid
Plasmaron
Quasi-isodynamic stellarator
Shockwave cosmology
Surface stress
Thermal energy
Toroidal solenoid
Wohlfarth Lectureship
Category:Physics by country


Аналогичным образом реализована работа с разнообразными другими API, имеющимися на разных сайтах. Где-то API является полностью открытым (как в Википедии), где-то вам потребуется зарегистрироваться и получить application id и какой-нибудь ключ для доступа к API, где-то попросят даже заплатить (например, автоматический поиск в Google). Всякий раз при использовании API вам придётся изучить его документацию, но это чаще всего проще, чем обрабатывать HTML-код. 

# Работа с API с помощью JSON¶

В примере выше для получения информации от API использовался формат XML. Помимо XML существует другой распространённый формат хранения и передачи структурированной информации, называющийся JSON. JSON расшифровывается как JavaScript Object Notation и изначально возник как подмножество языка JavaScript, используемое для описания объектов, но впоследствии стал использоваться и в других языках программирования, включая Python. Различные API могут поддерживать либо XML, либо JSON, либо и то, и другое, так что полезно научиться работать с обоими типами данных. Рассмотрим пример чтения данных из Википедии как в прошлый раз, но будем использовать формат JSON, API MediaWiki это позволяет.

In [15]:
url = "https://en.wikipedia.org/w/api.php"
myparams = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'json' #меняется только формат
}

g = requests.get(url, params=myparams)
g.ok

True

In [16]:
g.text

'{"batchcomplete":"","continue":{"cmcontinue":"page|44522e40322a503a4644010e01dc0d|1996857","continue":"-||"},"query":{"categorymembers":[{"pageid":22939,"ns":0,"title":"Physics"},{"pageid":844186,"ns":0,"title":"Modern physics"},{"pageid":1653925,"ns":100,"title":"Portal:Physics"},{"pageid":78053369,"ns":0,"title":"Bijel"},{"pageid":74985603,"ns":0,"title":"Edge states"},{"pageid":78147827,"ns":0,"title":"Electrostatic solitary wave"},{"pageid":78554064,"ns":0,"title":"History of the LED"},{"pageid":78751748,"ns":0,"title":"Missile lofting"},{"pageid":168907,"ns":0,"title":"Na\\u00efve physics"},{"pageid":78245824,"ns":0,"title":"Nottingham effect"}]}}'

Полученный результат похож на описание объекта в Python и смысл квадратных и фигурных скобок такой же. Правда, есть и отличия: например, в Python одинарные и двойные кавычки ничем не отличаются, а в JSON можно использовать только двойные. Мы видим, что полученный нами JSON представляет собой словарь, значения которого — строки или числа, а также списки или словари, значения которых в свою очередь также могут быть строками, числами, списками, словарями и т.д. То есть получается такая довольно сложная структура данных.

В данный момент тот факт, что перед нами сложная структура данных, видим только мы — с точки зрения Python, r.text это просто строка. Однако в модуле requests есть метод, позволяющий сразу выдать питоновский объект (словарь или список), если результат запроса возвращён в формате JSON. Так что нам не придётся использовать никакие дополнительные библиотеки.

In [17]:
q = g.json()
q

{'batchcomplete': '',
 'continue': {'cmcontinue': 'page|44522e40322a503a4644010e01dc0d|1996857',
  'continue': '-||'},
 'query': {'categorymembers': [{'pageid': 22939, 'ns': 0, 'title': 'Physics'},
   {'pageid': 844186, 'ns': 0, 'title': 'Modern physics'},
   {'pageid': 1653925, 'ns': 100, 'title': 'Portal:Physics'},
   {'pageid': 78053369, 'ns': 0, 'title': 'Bijel'},
   {'pageid': 74985603, 'ns': 0, 'title': 'Edge states'},
   {'pageid': 78147827, 'ns': 0, 'title': 'Electrostatic solitary wave'},
   {'pageid': 78554064, 'ns': 0, 'title': 'History of the LED'},
   {'pageid': 78751748, 'ns': 0, 'title': 'Missile lofting'},
   {'pageid': 168907, 'ns': 0, 'title': 'Naïve physics'},
   {'pageid': 78245824, 'ns': 0, 'title': 'Nottingham effect'}]}}

In [18]:
type(q)

dict

Видим, что это словарь. Содержательная информация хранится по ключу 'query'. А уже внутри есть ключ 'categorymembers', значением которого является список всех категорий. Каждая категория отображается в виде словаря, записями которого являются разные параметры категории (например, 'title' соответствует названию, а pageid — внутреннему идентификатору в системе).

In [19]:
q['query']['categorymembers']

[{'pageid': 22939, 'ns': 0, 'title': 'Physics'},
 {'pageid': 844186, 'ns': 0, 'title': 'Modern physics'},
 {'pageid': 1653925, 'ns': 100, 'title': 'Portal:Physics'},
 {'pageid': 78053369, 'ns': 0, 'title': 'Bijel'},
 {'pageid': 74985603, 'ns': 0, 'title': 'Edge states'},
 {'pageid': 78147827, 'ns': 0, 'title': 'Electrostatic solitary wave'},
 {'pageid': 78554064, 'ns': 0, 'title': 'History of the LED'},
 {'pageid': 78751748, 'ns': 0, 'title': 'Missile lofting'},
 {'pageid': 168907, 'ns': 0, 'title': 'Naïve physics'},
 {'pageid': 78245824, 'ns': 0, 'title': 'Nottingham effect'}]

In [20]:
type(q['query']['categorymembers'])

list

In [21]:
q['query']['categorymembers'][0]['title']

'Physics'

In [22]:
for cm in q['query']['categorymembers']:
    print(cm['title'])

Physics
Modern physics
Portal:Physics
Bijel
Edge states
Electrostatic solitary wave
History of the LED
Missile lofting
Naïve physics
Nottingham effect


_Источник: wiki.cs.hse.ru_